In [15]:
from datasets import load_dataset, Dataset, load_from_disk, concatenate_datasets
from MinHashDeduplication import Deduplicate
from RuleBasedFilter import RuleBasedFilter
import json
from tqdm import tqdm

In [17]:
raw_datasets = {
        "meta-math/MetaMathQA": ['train', 'query', None, False],
        "LDJnr/Capybara": ['train', 'conversation', 'conversation', False],
        "iamtarun/python_code_instructions_18k_alpaca": ['train', 'instruction', None, False],
        "open-r1/OpenR1-Math-220k": ['train', 'problem', None, False],
        "hkust-nlp/CodeIO-PyEdu-Reasoning": ['train', 'prompt', None, True],
    }

def run_filter(raw_datasets):
    for name, l in tqdm(raw_datasets.items()):
        streaming = l[3]
        if not streaming:
            ds = load_dataset(name, split=l[0])
        else:
            ds = []
            ds_stream = load_dataset(name, split=l[0], streaming=True)
            for i, sample in enumerate(ds_stream):
                if i >= 100000:
                    break
                ds.append(sample)
        
        dd_filter = Deduplicate(ds, l[1])
        ds_dd = dd_filter.run()
        rb_filter = RuleBasedFilter(ds_dd, l[2])
        ds_rb = rb_filter.run()
        ds_cleaned = Dataset.from_list(ds_rb)
        ds_cleaned.save_to_disk(f'./cleaned_datasets/{name}_cleaned')
        print(f'Dataset {name} cleaning completed. \n')

run_filter(raw_datasets)


  0%|          | 0/5 [00:00<?, ?it/s]

Original dataset size: 395000
Deduplicated dataset size: 50443
Original dataset size: 50443
Filtered dataset size: 50229


 20%|██        | 1/5 [05:14<20:59, 314.80s/it]

Dataset meta-math/MetaMathQA cleaning completed. 

Original dataset size: 16006
Deduplicated dataset size: 15056
Original dataset size: 15056
Filtered dataset size: 11803


 40%|████      | 2/5 [05:44<07:20, 146.83s/it]

...

Dataset iamtarun/python_code_instructions_18k_alpaca cleaning completed. 

Original dataset size: 93733
Deduplicated dataset size: 85158
Original dataset size: 85158
Filtered dataset size: 83834


 80%|████████  | 4/5 [12:54<03:35, 215.64s/it]

Dataset open-r1/OpenR1-Math-220k cleaning completed. 

Original dataset size: 100000
Deduplicated dataset size: 62670
Original dataset size: 62670
Filtered dataset size: 62659


100%|██████████| 5/5 [23:42<00:00, 284.40s/it]


Dataset hkust-nlp/CodeIO-PyEdu-Reasoning cleaning completed. 



In [ ]:
def dataset_concat(raw_datasets, samp=False):
    ds_list = []
    for name in tqdm(raw_datasets.keys()):
        if not samp:
            ds_list.append(load_from_disk(f'./cleaned_datasets/{name}_cleaned'))
        else:
            ds_list.append(load_from_disk(f'./cleaned_datasets/{name}_cleaned').select(range(2000)))

    ds_concat = concatenate_datasets(ds_list)
    ds_concat = ds_concat.add_column('index', range(len(ds_concat)))

    return ds_concat

samp_to_label = dataset_concat(raw_datasets, samp=True)
samp_to_label.save_to_disk('./cleaned_datasets/sample_to_label')



100%|██████████| 5/5 [00:00<00:00, 31.24it/s]


In [ ]:
def text_extract(sample):
    if sample['query']:
        text = sample['query'] + sample['response']
        dataset = 'metamath'
    elif sample['conversation']:
        text = []
        for t in sample['conversation']:
            text.append(" ".join(t.values()))
        text = " ".join(text)
        dataset = 'capybara'
    elif sample['instruction']:
        text = sample['prompt']
        dataset = 'code18k'
    elif sample['problem']:
        text = sample['problem'] + sample['solution'] + sample['answer']
        dataset = 'openmath'
    elif sample['turn_1']:
        try:
            text = sample['prompt'] + sample['turn_1'] + sample['feedback_1'] + sample['turn_2'] + sample['feedback_2']
        except:
            text = sample['prompt'] + sample['turn_1'] + sample['feedback_1']
        dataset = 'codeio'
    else:
        text = None
        dataset = None
    return {'text': f'{text}',
            'dataset': f'{dataset}'}


In [37]:
all_data = dataset_concat(raw_datasets)
all_data.save_to_disk('all_data')
all_data_shuffled6 = all_data.shuffle(seed=6)

texts = all_data_shuffled6.map(text_extract)
texts_extracted = texts.remove_columns([col for col in texts.column_names \
                                        if col != 'text' and col != 'dataset' and col != 'index'])
        
texts_extracted

Saving the dataset (11/11 shards): 100%|██████████| 225993/225993 [00:04<00:00, 50951.29 examples/s]


Dataset({
    features: ['index', 'text', 'dataset'],
    num_rows: 225993
})

In [35]:
with open('./jsonl/all_data_to_bert.jsonl', 'w', encoding='utf-8') as f:
    for text in tqdm(texts_extracted):
        f.write(json.dumps(text) + "\n")


100%|██████████| 225993/225993 [00:07<00:00, 28820.20it/s]
